# Process Google Timeline Data

In [12]:
# VARIABLES
json_file = "EZ_data/location-history.json"

In [1]:
import json
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from datetime import datetime

In [ ]:
def load_google_timeline(json_file):
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)
    
    locations = []
    for entry in data:
        if "visit" in entry:
            visit_info = entry["visit"]
            top_candidate = visit_info.get("topCandidate", {})

            if "placeLocation" in top_candidate:
                latlon_str = top_candidate["placeLocation"].replace("geo:", "")
                lat, lon = map(float, latlon_str.split(",")) 
                
                start_time = entry.get("startTime", "")
                end_time = entry.get("endTime", "")

                locations.append((lat, lon, start_time, end_time))

    return pd.DataFrame(locations, columns=["latitude", "longitude", "start_time", "end_time"])


df = load_google_timeline(json_file)
df.head()

,latitude,longitude,start_time,end_time
0,38.028028,-78.508516,2025-02-12T17:56:54.000-05:00,2025-02-13T10:44:48.307-05:00
1,38.031619,-78.510846,2025-02-13T10:51:39.463-05:00,2025-02-13T12:31:04.514-05:00
2,38.031619,-78.510846,2025-02-13T10:51:39.463-05:00,2025-02-13T11:03:44.537-05:00
3,38.028028,-78.508516,2025-02-13T12:36:02.657-05:00,2025-02-13T16:04:27.150-05:00
4,38.052815,-78.501257,2025-02-13T16:15:59.999-05:00,2025-02-13T17:02:51.018-05:00
